In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re, os, json
from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

class DocumentState(TypedDict):
    query: str
    documents: List[str]
    
def node_1(state: DocumentState) -> DocumentState:
    print("--node 1 (query update)")
    query  = state["query"]
    return {'query' : query}

def node_2(state: DocumentState) -> DocumentState:
    print("--node 2 (add documents)")
    return {"documents": ['doc1.pdf', 'doc2.pdf', 'doc3.pdf']}

def node_3(state: DocumentState) -> DocumentState:
    print("--node 3 (add more documents)")
    return {'documents': ['doc2.pdf', 'doc4.pdf', 'doc5.pdf']}

bulider = StateGraph(DocumentState)
bulider.add_node("node_1", node_1)
bulider.add_node("node_2", node_2)
bulider.add_node("node_3", node_3)

bulider.add_edge(START, "node_1")
bulider.add_edge("node_1", "node_2")
bulider.add_edge("node_2", "node_3")
bulider.add_edge("node_3", END)

graph = bulider.compile()


In [6]:
initial_state = {"query":"채식주의자를 위한 비건 음식을 추천해주세요", "documents": None}
final_state = graph.invoke(initial_state)
print("최종 상태:", final_state)

--node 1 (query update)
--node 2 (add documents)
--node 3 (add more documents)
최종 상태: {'query': '채식주의자를 위한 비건 음식을 추천해주세요', 'documents': ['doc2.pdf', 'doc4.pdf', 'doc5.pdf']}


In [7]:
from operator import add
from typing import Annotated, TypedDict
class ReducerState(TypedDict):
    query: str
    documents: Annotated[List[str], add]
    
def node_1(state: ReducerState) -> ReducerState:
    print("--node 1 (query update)")
    query  = state["query"]
    return {'query' : query}

def node_2(state: ReducerState) -> ReducerState:
    print("--node 2 (add documents)")
    return {"documents": ['doc1.pdf', 'doc2.pdf', 'doc3.pdf']}

def node_3(state: ReducerState) -> ReducerState:
    print("--node 3 (add more documents)")
    return {'documents': ['doc2.pdf', 'doc4.pdf', 'doc5.pdf']}

bulider = StateGraph(ReducerState)
bulider.add_node("node_1", node_1)
bulider.add_node("node_2", node_2)
bulider.add_node("node_3", node_3)

bulider.add_edge(START, "node_1")
bulider.add_edge("node_1", "node_2")
bulider.add_edge("node_2", "node_3")
bulider.add_edge("node_3", END)

graph = bulider.compile()

In [ ]:
initial_state = {"query":"채식주의자를 위한 비건 음식을 추천해주세요", "documents": []}
final_state = graph.invoke(initial_state)
print("최종 상태:", final_state)

--node 1 (query update)
--node 2 (add documents)
--node 3 (add more documents)
최종 상태: {'query': '채식주의자를 위한 비건 음식을 추천해주세요', 'documents': ['doc1.pdf', 'doc2.pdf', 'doc3.pdf', 'doc2.pdf', 'doc4.pdf', 'doc5.pdf']}


In [10]:
from typing import TypedDict, List, Annotated

def reduce_unique_dociments(left: list | None, right: list | None) -> list:
    """_summary_
    Combine two lists of documents, removing duplicates.
    Args:
        left (list | None): _description_
        right (list | None): _description_

    Returns:
        list: _description_
    """
    if not left:
        left = []
    if not right:
        right = []
    return list(set(left + right))

class CustomReducerState(TypedDict):
    query: str
    documents: Annotated[List[str], reduce_unique_dociments]
    
def node_1(state: CustomReducerState) -> CustomReducerState:
    print("--node 1 (query update)")
    query  = state["query"]
    return {'query' : query}

def node_2(state: CustomReducerState) -> CustomReducerState:
    print("--node 2 (add documents)")
    return {"documents": ['doc1.pdf', 'doc2.pdf', 'doc3.pdf']}

def node_3(state: CustomReducerState) -> CustomReducerState:
    print("--node 3 (add more documents)")
    return {'documents': ['doc2.pdf', 'doc4.pdf', 'doc5.pdf']}

bulider = StateGraph(CustomReducerState)
bulider.add_node("node_1", node_1)
bulider.add_node("node_2", node_2)
bulider.add_node("node_3", node_3)

bulider.add_edge(START, "node_1")
bulider.add_edge("node_1", "node_2")
bulider.add_edge("node_2", "node_3")
bulider.add_edge("node_3", END)

graph = bulider.compile()


In [11]:
initial_state = {"query":"채식주의자를 위한 비건 음식을 추천해주세요", "documents": []}
final_state = graph.invoke(initial_state)
print("최종 상태:", final_state)

--node 1 (query update)
--node 2 (add documents)
--node 3 (add more documents)
최종 상태: {'query': '채식주의자를 위한 비건 음식을 추천해주세요', 'documents': ['doc3.pdf', 'doc4.pdf', 'doc1.pdf', 'doc2.pdf', 'doc5.pdf']}


In [12]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class GraphState(TypedDict):
    messages : Annotated[list[AnyMessage], add_messages]
    


In [ ]:
from langgraph.graph import MessagesState
from typing import List
from langchain_core.documents import Document

class GraphState(MessagesState):
    #messages 키는 기본적으로 제공, 다른 키를 추가 가능.
    documents: List[Document]
    grade : float
    #무한루프 방지용
    num_generation : int

In [ ]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

embeddings_model = OllamaEmbeddings(model='bge-m3:latest')

vector_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="../../chroma_db"
)
llm = ChatOpenAI(model="gpt-4o-mini")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

system = """
You are a helpful assistant. Use the following context to answer the user's question:
[context]
{context}
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}")
])

retriever = vector_db.as_retriever(
    search_kwargs={"k":2}
)
#RunnablePassThrough()를 이용해서 retriever와 question에 동일한 값 전달
rag_chain = (
    {"context":retriever | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "채식주의자를 위한 메뉴를 추천해주세요."
response = rag_chain.invoke(query)

print(response)

채식주의자를 위한 메뉴로는 다음과 같은 옵션을 추천드립니다:

1. **가든 샐러드** - 가격: ₩12,000
   - 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 믹스 그린에 달콤한 체리 토마토, 오이, 당근을 더해 다양한 맛과 식감을 즐길 수 있습니다. 특제 발사믹 드레싱이 채소 본연의 맛을 살려줍니다.

2. **버섯 크림 수프** - 가격: ₩10,000
   - 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 

이 두 가지 메뉴는 채식주의자에게 적합하며, 건강하고 맛있는 선택이 될 것입니다.


In [22]:
def retrieve_and_respond(state: GraphState):
    last_human_message = state['messages'][-1]
    query = last_human_message.content
    
    retrieved_docs = retriever.invoke(query)
    response = rag_chain.invoke(query)
    return{
        "messages" :[AIMessage(content=response)],
        "documents": retrieved_docs
    }

In [23]:
from  pydantic import  BaseModel, Field
'''
스키마 정의
'''
class GradeResponse(BaseModel):
    "A score for answer"
    score: float = Field(..., ge=0, le=1, description="A score from 0 to 1, where 1 is perfect")
    explanation: str = Field(..., description="An explanation for the given score")
'''
답변의 품질을 평가하는 노드
'''
def grade_answer(state: GraphState):
    messages = state['messages']
    question = messages[-2].content
    answer = messages[-1].content
    context = format_docs(state['documents'])
    
    grading_system = """You are an expert grader.
    Grade the following answer based on its relevance and accuracy to the question, condsidering the given context.
    Provide a score from 0 to 1, where 1 is perfect, along wit an explatnation
    """
    grading_prompt = ChatPromptTemplate.from_messages({
        ("system", grading_system),
        ("human", "[Question]\n{question}\n\n[Context]\n{context}\n\n[Answer]\n{answer}\n\n[Grade]\n")
    })
    grading_chain = grading_prompt | llm.with_structured_output(schema=GradeResponse)
    
    grade_response = grading_chain.invoke({
        "question": question,
        "context": context,
        "answer": answer
    })
    
    num_generation = state.get('num_generation', 0)
    num_generation += 1
    return {"grade": grade_response.score, "num_generation":num_generation} 

In [24]:
from typing import Literal

def should_retry(state: GraphState) -> Literal["retrieve_and_respond", "generate"]:
    print("--Gradting--")
    print("Grade Score: ", state['grade'])
    
    if state["num_generation"] > 2:
        return "generate"
    
    if state["grade"] < 0.7:
        return "retrieve_and_respond"
    else:
        return "generate"

In [25]:
builder = StateGraph(GraphState)
builder.add_node("retrieve_and_respond", retrieve_and_respond)
builder.add_node("grade_answer", grade_answer)

builder.add_edge(START, "retrieve_and_respond")
builder.add_edge("retrieve_and_respond", "grade_answer")
builder.add_conditional_edges(
    "grade_answer",
    should_retry,
    {
        "retrieve_and_respond":"retrieve_and_respond",
        "generate": END
    }
)
graph = builder.compile()

In [26]:
shape = graph.get_graph().draw_mermaid()
print(shape)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve_and_respond(retrieve_and_respond)
	grade_answer(grade_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve_and_respond;
	retrieve_and_respond --> grade_answer;
	grade_answer -.-> retrieve_and_respond;
	grade_answer -. &nbsp;generate&nbsp; .-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [27]:
initial_state = {
    "messages": [HumanMessage(content="채식주의자를 위한 메뉴를 추천해주세요")],
}
final_state = graph.invoke(initial_state)
print("최종 상태: ", final_state)

--Gradting--
Grade Score:  1.0
최종 상태:  {'messages': [HumanMessage(content='채식주의자를 위한 메뉴를 추천해주세요', additional_kwargs={}, response_metadata={}, id='4243cfd7-9045-4a4a-9206-2502b016bd33'), AIMessage(content='채식주의자를 위한 메뉴로는 다음과 같은 옵션을 추천드립니다:\n\n1. **가든 샐러드** - ₩12,000\n   - 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이, 당근을 더해 다양한 맛과 식감을 즐길 수 있습니다. 특제 발사믹 드레싱이 채소 본연의 맛을 살려줍니다.\n\n2. **버섯 크림 수프** - ₩10,000\n   - 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다.\n\n3. **트러플 리조또** - ₩22,000\n   - 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했습니다.\n\n이 메뉴들은 모두 채식주의자에게 적합하며 맛도 뛰어난 요리들입니다.', additional_kwargs={}, response_metadata={}, id='25a6619b-3a28-4f81-aeb6-d27f64a607d3')], 'documents': [Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': '../../data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거

In [28]:
import gradio as gr
from typing import List, Tuple

# 예시 질문들
example_questions = [
    "채식주의자를 위한 메뉴를 추천해주세요.",
    "오늘의 스페셜 메뉴는 무엇인가요?",
    "파스타에 어울리는 음료는 무엇인가요?"
]

# 대답 함수 정의
def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))

        # 기존 채팅 기록에 사용자의 메시지를 추가 (최근 2개 대화만 사용)
        initial_state = {
            "messages": chat_history[-2:]+[HumanMessage(content=message)],  
        }

        # 메시지를 처리하고 최종 상태를 반환
        final_state = graph.invoke(initial_state)
        
        # 최종 상태에서 필요한 부분 반환 (예: 추천 메뉴 등)
        return final_state["messages"][-1].content
        
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."


# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# Gradio 앱 실행
demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


--Gradting--
Grade Score:  1.0
--Gradting--
Grade Score:  1.0
--Gradting--
Grade Score:  1.0
--Gradting--
Grade Score:  0.5
--Gradting--
Grade Score:  1.0


In [29]:
demo.close()

Closing server running on port: 7860
